In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [6]:
def extract_feature(file_name, mfcc, chroma, mel):
  with soundfile.SoundFile(file_name) as sound_file:
    X = sound_file.read(dtype="float32")
    sample_rate=sound_file.samplerate
    if chroma:
      stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
      mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
      result=np.hstack((result, mfccs))
    if chroma:
      chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
      result=np.hstack((result, chroma))
    if mel:
      mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
      result=np.hstack((result, mel))
  return result

In [3]:
emotions = {
  '01': 'anger',
  '02': 'clear',
  '03': 'loud',
  '04': 'neutral',
  '05': 'slow',
}

observed_emotions = ['anger', 'clear', 'loud', 'neutral', 'slow']

In [7]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("F:\\3Y-odd\\ML\\Project\\sph-stress\\speech_train\\*.sph"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[0]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [8]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [9]:
print((x_train.shape[0], x_test.shape[0]))

(75, 25)


In [10]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


MLP Classfier

In [24]:
model=MLPClassifier(alpha=0.01, batch_size=56, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [25]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=56, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [26]:
y_pred=model.predict(x_test)

In [27]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 64.00%


In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
cm

array([[2, 0, 0, 0, 1],
       [2, 4, 0, 0, 3],
       [1, 0, 4, 0, 0],
       [0, 0, 0, 4, 1],
       [0, 0, 0, 1, 2]], dtype=int64)

In [29]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()